0 get backscal

In [416]:
# get backscal in the pi files
from astropy.io import fits
import os
import numpy as np
from glob import glob

regname = 'reg12'

srcname = 'RGH80'
# srcnum = '9647'
homepath = glob(f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/RGH80/eckert/0105860101/{srcname}_{regname}')[0]
os.chdir(homepath)

inst_dict = {}
# every pixel length is 0.05 arcsec
for name in ['mos1S001', 'mos2S002', 'pnS003']:
    f = fits.open(f'{name}-back-{srcname}_{regname}.pi')
    print(f'''{name}: {np.round(f[1].header['BACKSCAL'] * (0.05/60) ** 2, 3)}''')
    inst_dict[name] = np.round(f[1].header['BACKSCAL'] * (0.05/60) ** 2, 3)

mos1S001: 128.681
mos2S002: 128.729
pnS003: 116.368


0.1 set bkg para

1 fit oot spec

In [276]:
# optional
print(f'''
ftgrouppha infile=pnS003-obj-oot-SDSSTG{srcnum}_{regname}.pi outfile=pnS003-obj-oot-SDSSTG{srcnum}_{regname}-grp.pi respfile=pnS003-SDSSTG{srcnum}_{regname}.rmf grouptype=opt
''')


ftgrouppha infile=pnS003-obj-oot-SDSSTG9647_reg1.pi outfile=pnS003-obj-oot-SDSSTG9647_reg1-grp.pi respfile=pnS003-SDSSTG9647_reg1.rmf grouptype=opt



In [417]:
print(f'''
abun aspl
statistic cstat

data pnS003-obj-oot-{srcname}_{regname}-grp.pi
res pnS003-{srcname}_{regname}.rmf
arf pnS003-{srcname}_{regname}.arf
backgrnd pnS003-back-{srcname}_{regname}.pi

ign 1.2-1.5
ign bad
ign **-0.3,7.0-**

mo const*const*(pow+pow+apec+apec)
''')


abun aspl
statistic cstat

data pnS003-obj-oot-RGH80_reg12-grp.pi
res pnS003-RGH80_reg12.rmf
arf pnS003-RGH80_reg12.arf
backgrnd pnS003-back-RGH80_reg12.pi

ign 1.2-1.5
ign bad
ign **-0.3,7.0-**

mo const*const*(pow+pow+apec+apec)



In [418]:
print(f'''
new 1 {inst_dict['pnS003']}
new 2 1
free 1,2
''')


new 1 116.368
new 2 1
free 1,2



In [109]:
fit

NameError: name 'fit' is not defined

In [ ]:
cpd /xs
setp energy
pl ldat rat

In [ ]:
save all oot.xcm

rm oot.log
log >oot.log
sho par 
log done
exit

2 load the data

In [419]:
print(f'''
data 1:1 mos1S001-obj-{srcname}_{regname}-grp.pi
data 2:2 mos2S002-obj-{srcname}_{regname}-grp.pi
data 3:3 pnS003-obj-{srcname}_{regname}-grp.pi

res 1 mos1S001-{srcname}_{regname}.rmf
res 2 mos2S002-{srcname}_{regname}.rmf
res 3 pnS003-{srcname}_{regname}.rmf

arf 1 mos1S001-{srcname}_{regname}.arf
arf 2 mos2S002-{srcname}_{regname}.arf
arf 3 pnS003-{srcname}_{regname}.arf

# backgrnd 1 mos1S001-back-{srcname}_{regname}.pi
# backgrnd 2 mos2S002-back-{srcname}_{regname}.pi
# backgrnd 3 pnS003-back-{srcname}_{regname}.pi

setp energy
statistic cstat
abun aspl

ign 1:0.0-0.5,7.0-** 
ign 2:0.0-0.5,7.0-**
ign 3:0.0-0.5,7.0-**

# ign instrumental lines
ign 1:1.2-1.85
ign 2:1.2-1.85
ign 3:1.2-1.55
ign bad

cpd /xs
pl ldat
''')


data 1:1 mos1S001-obj-RGH80_reg12-grp.pi
data 2:2 mos2S002-obj-RGH80_reg12-grp.pi
data 3:3 pnS003-obj-RGH80_reg12-grp.pi

res 1 mos1S001-RGH80_reg12.rmf
res 2 mos2S002-RGH80_reg12.rmf
res 3 pnS003-RGH80_reg12.rmf

arf 1 mos1S001-RGH80_reg12.arf
arf 2 mos2S002-RGH80_reg12.arf
arf 3 pnS003-RGH80_reg12.arf

# backgrnd 1 mos1S001-back-RGH80_reg12.pi
# backgrnd 2 mos2S002-back-RGH80_reg12.pi
# backgrnd 3 pnS003-back-RGH80_reg12.pi

setp energy
statistic cstat
abun aspl

ign 1:0.0-0.5,7.0-** 
ign 2:0.0-0.5,7.0-**
ign 3:0.0-0.5,7.0-**

# ign instrumental lines
ign 1:1.2-1.85
ign 2:1.2-1.85
ign 3:1.2-1.55
ign bad

cpd /xs
pl ldat



3 set mdl

In [ ]:
mo const*const*(apec + tbabs*(apec+apec+pow))

In [420]:
print(f'''
# extract region area
new 1 {inst_dict['mos1S001']}
new 18 {inst_dict['mos2S002']}
new 35 {inst_dict['pnS003']}
free 1,18,35

# inst variance
new 2 1
new 19 1
new 36 1
free 2,19,36

# LHB
new 3 0.11
new 6 8.416E-07
free 3,6

# nH
new 7 0.0131
free 7

# GH_hard
# not always need to consider
new 8 0.191207
new 11 2.01079E-06
free 8,11


# ICM
new 12 0.2
new 13 0.3
new 14 0.037
new 15 2e-6
free 14
thaw 12,13,15

# cxb
new 16 1.46
new 17 1.00448E-06
free 16,17
# always needs to be frozen 
''')



# extract region area
new 1 128.681
new 18 128.729
new 35 116.368
free 1,18,35

# inst variance
new 2 1
new 19 1
new 36 1
free 2,19,36

# LHB
new 3 0.11
new 6 8.416E-07
free 3,6

# nH
new 7 0.0131
free 7

# GH_hard
# not always need to consider
new 8 0.191207
new 11 2.01079E-06
free 8,11


# ICM
new 12 0.2
new 13 0.3
new 14 0.037
new 15 2e-6
free 14
thaw 12,13,15

# cxb
new 16 1.46
new 17 1.00448E-06
free 16,17
# always needs to be frozen 



6 set oot model

In [421]:
print(f'''
res 2:3 pnS003-{srcname}_{regname}.rmf
arf 2:3 pnS003-{srcname}_{regname}.arf
backgrnd 2:3 pnS003-back-{srcname}_{regname}.pi

mo 2:oot const*const*(pow + pow + apec + apec)
''')


res 2:3 pnS003-RGH80_reg12.rmf
arf 2:3 pnS003-RGH80_reg12.arf
backgrnd 2:3 pnS003-back-RGH80_reg12.pi

mo 2:oot const*const*(pow + pow + apec + apec)



In [422]:
import re

with open(f'oot.log', 'r') as file:
    lines = file.readlines()

value_data = []

# Iterate through the lines and extract the Value column using regex
for line in lines:
    if line.startswith('# '):
        if re.search(r'\d', line):  # Check if the line contains a number
            columns = line.split()
            if 'frozen' in line:
                value = columns[-2]
            else:
                value = columns[-3]
            value_data.append(value)

# Print the Value data
for i, value in enumerate(value_data):
    print(f'new oot:{int(i+1)} {value}' )
print(f'free oot:1-14')


new oot:1 116.368
new oot:2 1.00000
new oot:3 9.50000
new oot:4 1.07794E-09
new oot:5 1.77479
new oot:6 2.33262E-06
new oot:7 0.201209
new oot:8 1.00000
new oot:9 0.0
new oot:10 1.64413E-06
new oot:11 0.717842
new oot:12 1.00000
new oot:13 0.0
new oot:14 1.20215E-06
free oot:1-14


In [ ]:
new oot:2 0.063
free oot:2

5 set spf mdl

In [423]:
print(f'''
resp 3:1 ../mos1-diag.rsp 
resp 3:2 ../mos2-diag.rsp 
resp 3:3 ../pn-diag.rsp 

# backgrnd 3:1 mos1S001-back-{srcname}_{regname}.pi	
# backgrnd 3:2 mos2S002-back-{srcname}_{regname}.pi
# backgrnd 3:3 pnS003-back-{srcname}_{regname}.pi

model 3:spf const*const*(bknpow)
''')


resp 3:1 ../mos1-diag.rsp 
resp 3:2 ../mos2-diag.rsp 
resp 3:3 ../pn-diag.rsp 

# backgrnd 3:1 mos1S001-back-RGH80_reg12.pi	
# backgrnd 3:2 mos2S002-back-RGH80_reg12.pi
# backgrnd 3:3 pnS003-back-RGH80_reg12.pi

model 3:spf const*const*(bknpow)



In [415]:
print(f'''
# set area const
new spf:1=1
new spf:7=18
new spf:13=35
free spf:1,7,13

new spf:2=2
new spf:8=19
new spf:14=36

# set bknpow index
new spf:3 0.7
new spf:9 0.7
new spf:15 0.7

new spf:5 0.7
new spf:11 0.7
new spf:17 0.7
free spf:3,9,15,5,11,17

# set breakE
free spf:4,10,16

# estimate spf norm
new spf:6 4.78172E-06
new spf:12 6.60607E-06
new spf:18 7.1487E-06
free spf:6,12,18
''')


# set area const
new spf:1=1
new spf:7=18
new spf:13=35
free spf:1,7,13

new spf:2=2
new spf:8=19
new spf:14=36

# set bknpow index
new spf:3 0.7
new spf:9 0.7
new spf:15 0.7

new spf:5 0.7
new spf:11 0.7
new spf:17 0.7
free spf:3,9,15,5,11,17

# set breakE
free spf:4,10,16

# estimate spf norm
new spf:6 4.78172E-06
new spf:12 6.60607E-06
new spf:18 7.1487E-06
free spf:6,12,18



In [ ]:
fit

In [ ]:
cpd /xs
setp energy
pl ldat rat

In [ ]:
sho fre

In [ ]:
save all fit_pnmos.xcm
exit